# **Mount the Drive and change the working directory:**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/MSC Thesis

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/MSC Thesis


# Import Required Modules

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import sklearn
from sklearn.feature_selection import mutual_info_regression

predictors = pd.read_csv('pred_inputs.csv')
predict_these = np.unique(predictors['INSTRUMENT'])

# Functions for preprocessing and Feature Generation

In [0]:
def from_csv(instrument):
    df = pd.read_csv('Data/' + instrument + '.csv')
    df['date'] = pd.to_datetime(df['date']).dt.date
    df.set_index('date',inplace=True)
    df = df.sort_values(by = 'date')
    df.columns = [instrument]
    return df
  
def preprocessing(commodity):
    #####################################################################################
    # This ensures chronological order (some time series are NOT in order)              #
    #####################################################################################
    
    prices_df = db.get_instrument_data(commodity)
    prices_df = prices_df.sort_values(by = 'date')
    prices_df.columns = [commodity]
    return prices_df
  
def get_predictors(commodity, predictors):
    ####################################################################
    # This returns the df containing the values of the commodity to be #
    # predicted as well as that of its predictors.                     #
    #                                                                  #
    # Args:                                                            #
    # 1) commodity: name of the commodity                              #
    # 2) predictors: the dataframe sent by Michael Button              #
    ####################################################################
    
    # get a list of all its predictors
    Pred = list(predictors.loc[predictors['INSTRUMENT'] == commodity]['INPUT'])
    Pred = [p for p in Pred if p != commodity]
    
    # Initialize the dataframe with the commodity we want to predict
    DF = from_csv( commodity )
    
    # Keep adding the predictors
    for predictor in Pred:
        temp_df = from_csv( predictor )
        DF = DF.join(temp_df)
    return DF
  
def price_to_returns(prices_df, diff):
    ###############################################################
    # This changes the price data to 5 day log difference.        #
    # If X has negative values, use the following scheme:         # 
    #          X := X - min(X) + 1                                #
    ###############################################################
    
    # forward fill
    prices_df.fillna(method='ffill', inplace = True)
    
    returns_df = prices_df.copy()
    for colname in prices_df.columns:
        temp = prices_df.loc[:,colname]
        
        if np.min(temp) <= 0:
            temp = temp - np.min(temp) + 1
        returns_df[colname] = np.log(temp) - np.log(temp.shift(diff))

    # drop rows with NaN
    returns_df.dropna(inplace = True)
    return returns_df
  
  
def feature_generation(metal, time_pred = [1,5,22]):
  metal = metal + '_lme_prices'
  Metal_DF = get_predictors(metal, predictors)
  # remove comex, since comex is not 
  Metal_DF = Metal_DF[[col for col in list(Metal_DF.columns) if 'comex' not in col]]
  
  # 1, 5, 22 days
  # log difference L
  LD = [price_to_returns(Metal_DF, t_pred) for t_pred in time_pred]

  # EWMA of L
  # EWMA = [L.ewm(span = horizon).mean() for horizon, L in zip(time_pred, LD)]
  EWMA = [LD[0].ewm(halflife = horizon).mean() for horizon in time_pred ]

  # EWMV of L
  # 1. calculate expanding window mean for the returns (1 day) 
  # 2. subtract rolling mean and take square
  EM = LD[0].expanding(2).mean() 
  EWMV = [((LD[0] - EM)**2).ewm(span = horizon).mean()**0.5 for horizon in time_pred ]
  
  # rename columns
  for horizon, ld, ewma, ewmv in zip(time_pred, LD, EWMA, EWMV):
    ld.columns = [col + '_LD_' + str(horizon) for col in ld.columns]
    ewma.columns = [col + '_EWMA_' + str(horizon) for col in ewma.columns]
    ewmv.columns = [col + '_EWMV_' + str(horizon) for col in ewmv.columns]
    
  # merge together 
  ALL_FEATURES = pd.concat([pd.concat(DFS, axis = 1, sort=True) for DFS in [LD, EWMA, EWMV]], axis = 1)
  ALL_FEATURES_columns = list(ALL_FEATURES.columns)
  ALL_FEATURES_columns.sort()
  ALL_FEATURES = ALL_FEATURES.loc[:,ALL_FEATURES_columns]
  ALL_FEATURES.dropna(inplace = True)
  
  return ALL_FEATURES



def feature_extraction(DF, column_name, lag):
  # This extracts the relevant predictors for the column_name
  # threshold is the % of predictors that we want to include
  # Mutual Information
  DF_metal = DF.loc[:,[column_name]]
  MI_table = np.zeros(DF.shape[1])
  for i in range(DF.shape[1]):
      MI = sklearn.feature_selection.mutual_info_regression(DF_metal, DF.iloc[:,i])
      MI_table[i] = MI
  
  tol = 1e-3
  MI_table_2 = MI_table[MI_table > tol]
  out, bins = pd.qcut(MI_table_2, [0.9, 1], retbins=True)
  selected_cols = list(MI_table > bins[-2])
  
  DF = DF.loc[:,selected_cols].shift(periods = lag)
  DF.columns = [col + '_lag' for col in DF.columns]
  DF = pd.concat([DF_metal, DF], axis = 1, sort = True).dropna()
  time_column = pd.DataFrame(index = DF.index, 
                             data = np.arange(DF.shape[0])*0.01 , 
                             columns = ['time'] )
  DF = pd.concat([DF, time_column], axis = 1)
  # add time column
  
  
  return(DF)


# Import GPytorch

In [0]:
!pip install gpytorch
import torch
import gpytorch 

     |████████████████████████████████| 215kB 2.9MB/s 
  Created wheel for gpytorch: filename=gpytorch-0.3.5-py2.py3-none-any.whl size=349719 sha256=01e8bfcfa41216e255f7c261b3b54b2639837e48dff9c08b1454bd74fcc7a5b6
  Stored in directory: /root/.cache/pip/wheels/d6/31/88/c43a94e0073a54056ac663366f2195de36535b38a81a378196
Successfully built gpytorch


# Get Gpytorch model ExactGPModel/MultitaskGPModel

In [0]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(kernel)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
      

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, rank, kernel):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks= train_y.shape[0]
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            kernel, num_tasks= train_y.shape[0], rank=rank
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)



# Aluminium and Copper - Same Time Horizons



1.   T+1
2.   T+5
3.   T+22



In [0]:
def MGP_train(X_train, X_test, Y_train, Y_test, kernel, rank):
  likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks= Y_train.shape[0])
  model = MultitaskGPModel(X_train, Y_train, likelihood, rank = rank, 
                           kernel = kernel(ard_num_dims = X_train.shape[1]))

  model.train()
  likelihood.train()

  # Use the adam optimizer
  optimizer = torch.optim.Adam([
      {'params': model.parameters()},  # Includes GaussianLikelihood parameters
  ], lr=0.1)

  # "Loss" for GPs - the marginal log likelihood
  mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

  n_iter = 100
  for i in range(n_iter):
      optimizer.zero_grad()
      output = model(X_train)
      loss = -mll(output, Y_train)
      loss.backward()
      optimizer.step()
  model.eval()
  likelihood.eval()

  with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions = likelihood(model(X_test))
    mean = predictions.mean

  mse = (mean - Y_test.t() ).numpy()**2
  direction = (torch.sign(mean) == torch.sign(Y_test.t())).numpy()

  return(mse, direction)

def GP_train(X_train, X_test, Y_train, Y_test, kernel):
  likelihood = gpytorch.likelihoods.GaussianLikelihood()
  model = ExactGPModel(X_train, Y_train, likelihood, kernel(ard_num_dims = X_train.shape[1]))
  model.train()
  likelihood.train()

  # Use the adam optimizer
  optimizer = torch.optim.Adam([
      {'params': model.parameters()},  # Includes GaussianLikelihood parameters
  ], lr=0.1)

  # "Loss" for GPs - the marginal log likelihood
  mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
  training_iter = 100
  for l in range(training_iter):
      optimizer.zero_grad()
      output = model(X_train)
      loss = -mll(output, Y_train)
      loss.backward()
      optimizer.step()

  # Get into evaluation (predictive posterior) mode
  model.eval()
  likelihood.eval()

  # Make predictions by feeding model through likelihood
  with torch.no_grad(), gpytorch.settings.fast_pred_var():
      predictions = likelihood(model(X_test))
      mean = predictions.mean

  mse = (mean - Y_test.t() ).numpy()**2
  direction = (torch.sign(mean) == torch.sign(Y_test.t())).numpy()


  return(mse, direction)

def Across_Metals(window, horizon, Target_Variables, kernel, trials):

  # metal is either 'al' or 'cu'
  # window is the length of data before prediction
  # Horizons is a list of numbers (e.g. [1,5,22])
  # Target_Variables is a list of strings 
  #      (e.g. ['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'])
  # kernel is the kernel that we will use 
  # trials (self explanatory)
  # n_test (number of test points per trial)
  
  I = len(Target_Variables)
  
  
  # this
  MSE_MGP = np.zeros((I,I,trials)) 
  DIR_MGP = np.zeros((I,I,trials))  
  
  MSE_GP = np.zeros((I,trials))
  DIR_GP = np.zeros((I,trials))
  
  MSE_BM = np.zeros((I,trials))
  
  # this contains the list of dataframes 
  DF_list = []
  for TV_index,TV in enumerate(Target_Variables):
    # horizon and target variable
    DF = feature_generation(TV[0:2], time_pred = [1,5,22])
    DF = feature_extraction(DF, TV, horizon)
    DF_list.append(DF)
  
  # now join the DFs in DF_list, and remove duplicate columns
  DF = pd.concat(DF_list, axis = 1, sort = True).dropna()
  DF = DF.loc[:,~DF.columns.duplicated()]
  Y = DF.loc[:,Target_Variables]
  X = DF.drop(Target_Variables, axis = 1)
  
  ard_param = X.shape[1]

  
  for t in range(trials):
#     if t%10 == 0:
#       print(t)
    START = int(np.random.choice(np.arange(len(Y) - 200),1) )
    END = START + window
    X_train = torch.tensor( X.iloc[START:END, :].values , dtype = torch.float32)
    X_test = torch.tensor( X.iloc[[(END+horizon-1)], :].values , dtype = torch.float32)
    Y_train = torch.tensor( Y.iloc[START:END, :].values.T.squeeze() , dtype = torch.float32)
    Y_test = torch.tensor( Y.iloc[[(END+horizon-1)], :].values.T.squeeze() , dtype = torch.float32)
    
    for rank in range(Y_train.shape[0]):
      mse, direction = MGP_train(X_train, X_test, Y_train, Y_test, kernel, rank+1)
      
      MSE_MGP[:,rank, t] = mse
      DIR_MGP[:,rank, t] = direction
    
    
    ###################################################
    for TV_index, TV in enumerate(Target_Variables):
      Y_train_temp, Y_test_temp = Y_train[TV_index,:], Y_test[TV_index]
      
      mse, direction = GP_train(X_train, X_test, Y_train_temp, Y_test_temp, kernel)
      
      MSE_GP[TV_index,t] = mse
      DIR_GP[TV_index,t] = direction
      MSE_BM[TV_index,t] = Y_test_temp**2
      
      
      
      
  return MSE_MGP, DIR_MGP, MSE_GP ,DIR_GP, MSE_BM


# Rank Experiments

In [0]:
horizons = [1,5,22]
window = 100
kernel = gpytorch.kernels.RBFKernel
Joint_Predictions = [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
                     ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  MSE_MGP, DIR_MGP, MSE_GP ,DIR_GP, MSE_BM = Across_Metals(window, horizon, Target_Variables, kernel, trials=200)
  print('T+',horizon)
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':')
    
    
    for rank in range(MSE_MGP.shape[1]):
      print('RANK ',rank+1)
      print('MGP MSE, ',np.mean(MSE_MGP[tv, rank,:] ))
      print('MGP DIR ',np.mean(DIR_MGP[tv, rank,:] ) )
      print(' ')
    
    
    print('GP MSE', np.mean(MSE_GP[tv,:] ))
    print('GP DIR', np.mean(DIR_GP[tv,:] ))
    print(' ')
    print('Benchmark', np.mean(MSE_BM[tv,:] ) )
    print(' ')
  print('######')

T+ 1
AL :
RANK  1
MGP MSE,  0.00019008603663959734
MGP DIR  0.565
 
RANK  2
MGP MSE,  0.00023172640151968355
MGP DIR  0.51
 
GP MSE 0.00020222764365932734
GP DIR 0.575
 
Benchmark 0.00019701978318443115
 
CU :
RANK  1
MGP MSE,  0.0004194529094075805
MGP DIR  0.47
 
RANK  2
MGP MSE,  0.0003829185752585784
MGP DIR  0.54
 
GP MSE 0.00035720578924392524
GP DIR 0.47
 
Benchmark 0.0003284915483078998
 
######
T+ 5
AL :
RANK  1
MGP MSE,  0.0013858802747719779
MGP DIR  0.525
 
RANK  2
MGP MSE,  0.00143289286918737
MGP DIR  0.57
 
GP MSE 0.0017173348947960498
GP DIR 0.485
 
Benchmark 0.0010661773309052869
 
CU :
RANK  1
MGP MSE,  0.0019158144503034435
MGP DIR  0.52
 
RANK  2
MGP MSE,  0.0020209886701481137
MGP DIR  0.515
 
GP MSE 0.0021319597031030126
GP DIR 0.46
 
Benchmark 0.001355903968759513
 
######
T+ 22
AL :
RANK  1
MGP MSE,  0.0424535416635544
MGP DIR  0.505
 
RANK  2
MGP MSE,  0.05448761384785941
MGP DIR  0.5
 
GP MSE 0.008895143228465319
GP DIR 0.57
 
Benchmark 0.003399183035746205
 


# Skeleton Version - only the predicted time series

In [0]:
def Across_Metals_Skeleton(window, horizon, Target_Variables, kernel, trials):

  # metal is either 'al' or 'cu'
  # window is the length of data before prediction
  # Horizons is a list of numbers (e.g. [1,5,22])
  # Target_Variables is a list of strings 
  #      (e.g. ['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'])
  # kernel is the kernel that we will use 
  # trials (self explanatory)
  # n_test (number of test points per trial)
  
  I = len(Target_Variables)
  
  
  # this
  MSE_MGP = np.zeros((I,I,trials)) 
  DIR_MGP = np.zeros((I,I,trials))  
  
  MSE_GP = np.zeros((I,trials))
  DIR_GP = np.zeros((I,trials))
  
  MSE_BM = np.zeros((I,trials))
  
  # this contains the list of dataframes 
  DF_list = []
  for TV_index,TV in enumerate(Target_Variables):
    # horizon and target variable
    DF = feature_generation(TV[0:2], time_pred = [1,5,22])
    DF = feature_extraction(DF, TV, horizon)
    DF_list.append(DF)
  
  # now join the DFs in DF_list, and remove duplicate columns
  DF = pd.concat(DF_list, axis = 1, sort = True).dropna()
  DF = DF.loc[:,~DF.columns.duplicated()]
  Y = DF.loc[:,Target_Variables]
  X = Y.shift(horizon).iloc[horizon:,:]
  Y = Y.iloc[horizon:,:]
  
  ard_param = X.shape[1]

  
  for t in range(trials):
#     if t%10 == 0:
#       print(t)
    START = int(np.random.choice(np.arange(len(Y) - 200),1) )
    END = START + window
    X_train = torch.tensor( X.iloc[START:END, :].values , dtype = torch.float32)
    X_test = torch.tensor( X.iloc[[(END+horizon-1)], :].values , dtype = torch.float32)
    Y_train = torch.tensor( Y.iloc[START:END, :].values.T.squeeze() , dtype = torch.float32)
    Y_test = torch.tensor( Y.iloc[[(END+horizon-1)], :].values.T.squeeze() , dtype = torch.float32)
    
    
    for rank in range(Y_train.shape[0]):
      mse, direction = MGP_train(X_train, X_test, Y_train, Y_test, kernel, rank+1)
      
      MSE_MGP[:,rank, t] = mse
      DIR_MGP[:,rank, t] = direction
    
    
    ###################################################
    for TV_index, TV in enumerate(Target_Variables):
      Y_train_temp, Y_test_temp = Y_train[TV_index,:], Y_test[TV_index]
      
      mse, direction = GP_train(X_train, X_test, Y_train_temp, Y_test_temp, kernel)
      
      MSE_GP[TV_index,t] = mse
      DIR_GP[TV_index,t] = direction
      MSE_BM[TV_index,t] = Y_test_temp**2
      
      
      
      
  return MSE_MGP, DIR_MGP, MSE_GP ,DIR_GP, MSE_BM


In [0]:
window = 100
horizons = [1,5,22]
kernel = gpytorch.kernels.RBFKernel
Joint_Predictions = [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
                     ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  MSE_MGP, DIR_MGP, MSE_GP ,DIR_GP, MSE_BM = Across_Metals_Skeleton(window, horizon, Target_Variables, kernel, trials=200)
  print('T+',horizon)
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':')
    
    
    for rank in range(MSE_MGP.shape[1]):
      print('RANK ',rank+1)
      print('MGP MSE, ',np.mean(MSE_MGP[tv, rank,:] ))
      print('MGP DIR ',np.mean(DIR_MGP[tv, rank,:] ) )
      print(' ')
    
    
    print('GP MSE', np.mean(MSE_GP[tv,:] ))
    print('GP DIR', np.mean(DIR_GP[tv,:] ))
    print(' ')
    print('Benchmark', np.mean(MSE_BM[tv,:] ) )
    print(' ')
  print('######')

T+ 1
AL :
RANK  1
MGP MSE,  0.0001979905890562783
MGP DIR  0.54
 
RANK  2
MGP MSE,  0.00020101346046870873
MGP DIR  0.515
 
GP MSE 0.00019370760414052056
GP DIR 0.505
 
Benchmark 0.0001917812306487754
 
CU :
RANK  1
MGP MSE,  0.00029452421910978186
MGP DIR  0.52
 
RANK  2
MGP MSE,  0.00029911516960177574
MGP DIR  0.535
 
GP MSE 0.0003045833126363018
GP DIR 0.525
 
Benchmark 0.00030284872858656
 
######
T+ 5
AL :
RANK  1
MGP MSE,  0.0010818835600337117
MGP DIR  0.525
 
RANK  2
MGP MSE,  0.001109269520475431
MGP DIR  0.48
 
GP MSE 0.0011064866395962625
GP DIR 0.525
 
Benchmark 0.0010218701675537645
 
CU :
RANK  1
MGP MSE,  0.001565278196574269
MGP DIR  0.52
 
RANK  2
MGP MSE,  0.001602432421716946
MGP DIR  0.53
 
GP MSE 0.0014624442494745615
GP DIR 0.45
 
Benchmark 0.0013450213391649867
 
######
T+ 22
AL :
RANK  1
MGP MSE,  0.007619898649049173
MGP DIR  0.545
 
RANK  2
MGP MSE,  0.0129248776261457
MGP DIR  0.535
 
GP MSE 0.007362920552477163
GP DIR 0.525
 
Benchmark 0.004855046705732775


In [0]:
window = 100
horizons = [22]#[1,5,22]
kernel = gpytorch.kernels.RBFKernel
Joint_Predictions = [['al_lme_prices_LD_22', 'cu_lme_prices_LD_22']]
# [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
#                      ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
#                      ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  MSE_MGP, DIR_MGP, MSE_GP ,DIR_GP, MSE_BM = Across_Metals_Skeleton(window, horizon, Target_Variables, kernel, trials=1000)
  print('T+',horizon)
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':')
    
    
    for rank in range(MSE_MGP.shape[1]):
      print('RANK ',rank+1)
      print('MGP MSE, ',np.mean(MSE_MGP[tv, rank,:] ))
      print('MGP DIR ',np.mean(DIR_MGP[tv, rank,:] ) )
      print(' ')
    
    
    print('GP MSE', np.mean(MSE_GP[tv,:] ))
    print('GP DIR', np.mean(DIR_GP[tv,:] ))
    print(' ')
    print('Benchmark', np.mean(MSE_BM[tv,:] ) )
    print(' ')
  print('######')

T+ 22
AL :
RANK  1
MGP MSE,  0.007360535399634894
MGP DIR  0.547
 
RANK  2
MGP MSE,  0.007116665514774421
MGP DIR  0.549
 
GP MSE 0.005832327249364647
GP DIR 0.555
 
Benchmark 0.004375411421404854
 
CU :
RANK  1
MGP MSE,  0.009845903844722035
MGP DIR  0.49
 
RANK  2
MGP MSE,  0.008897204518264914
MGP DIR  0.492
 
GP MSE 0.010018054696468695
GP DIR 0.486
 
Benchmark 0.007187018584617185
 
######


In [0]:
def MGP_train_CI(X_train, X_test, Y_train, Y_test, kernel, rank):
  likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks= Y_train.shape[0])
  model = MultitaskGPModel(X_train, Y_train, likelihood, rank = rank, 
                           kernel = kernel(ard_num_dims = X_train.shape[1]))

  model.train()
  likelihood.train()

  # Use the adam optimizer
  optimizer = torch.optim.Adam([
      {'params': model.parameters()},  # Includes GaussianLikelihood parameters
  ], lr=0.1)

  # "Loss" for GPs - the marginal log likelihood
  mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

  n_iter = 100
  for i in range(n_iter):
      optimizer.zero_grad()
      output = model(X_train)
      loss = -mll(output, Y_train)
      loss.backward()
      optimizer.step()
  model.eval()
  likelihood.eval()

  with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions = likelihood(model(X_test))
    mean = predictions.mean
    
  lower, upper = predictions.confidence_region()
#   print(lower, upper)
#   print(Y_test)
  Included = (upper.squeeze() > Y_test).numpy() * (lower.squeeze() < Y_test).numpy()
  
  
  
  lower = lower.numpy()
  upper = upper.numpy()
  Interval = (upper - lower).squeeze()
  print(Interval)
  return (Interval, Included)
  
  
#   mse = (mean - Y_test.t() ).numpy()**2
#   direction = (torch.sign(mean) == torch.sign(Y_test.t())).numpy()

#   return(mse, direction)

def Across_Metals_CI(window, horizon, Target_Variables, kernel, trials):

  # metal is either 'al' or 'cu'
  # window is the length of data before prediction
  # Horizons is a list of numbers (e.g. [1,5,22])
  # Target_Variables is a list of strings 
  #      (e.g. ['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'])
  # kernel is the kernel that we will use 
  # trials (self explanatory)
  # n_test (number of test points per trial)
  
  I = len(Target_Variables)
  
  
  # this
  #number of TV/Rank/Trials 
  CI_MGP = np.zeros((I,trials)) 
  Included_MGP = np.zeros((I,trials))  
  
  # this contains the list of dataframes 
  DF_list = []
  for TV_index,TV in enumerate(Target_Variables):
    # horizon and target variable
    DF = feature_generation(TV[0:2], time_pred = [1,5,22])
    DF = feature_extraction(DF, TV, horizon)
    DF_list.append(DF)
  
  # now join the DFs in DF_list, and remove duplicate columns
  DF = pd.concat(DF_list, axis = 1, sort = True).dropna()
  DF = DF.loc[:,~DF.columns.duplicated()]
  
  Y = DF.loc[:,Target_Variables]
  X = Y.shift(horizon).iloc[horizon:,:]
  Y = Y.iloc[horizon:,:]
  
#   Y = DF.loc[:,Target_Variables]
#   X = DF.drop(Target_Variables, axis = 1)
  
  ard_param = X.shape[1]

  Chosen = np.random.choice(np.arange(len(Y) - 200) , trials ,replace = False)
  
  for t in range(trials):

    START = Chosen[t]
    END = START + window
    X_train = torch.tensor( X.iloc[START:END, :].values , dtype = torch.float32)
    X_test = torch.tensor( X.iloc[[(END+horizon-1)], :].values , dtype = torch.float32)
    Y_train = torch.tensor( Y.iloc[START:END, :].values.T.squeeze() , dtype = torch.float32)
    Y_test = torch.tensor( Y.iloc[[(END+horizon-1)], :].values.T.squeeze() , dtype = torch.float32)
    
    Interval, Included = MGP_train_CI(X_train, X_test, Y_train, Y_test, kernel, rank = 2)
    
    if np.sum(np.isnan(Interval)) > 0:
      CI_MGP[:,t] = np.nan
      Included_MGP[:,t] = np.nan
    else:
      CI_MGP[:,t] = Interval
      Included_MGP[:,t] = Included
  
  
  return CI_MGP, Included_MGP


In [0]:
window = 100
horizons = [1,5,22]
kernel = gpytorch.kernels.RBFKernel
Joint_Predictions = [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
                     ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  CI_MGP, Included_MGP = Across_Metals_CI(window, horizon, Target_Variables, kernel, trials=100)
  print('T+',horizon)
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':')
    
    
    print('MGP CI, ',np.nanmean(CI_MGP[tv, :] ))
    print('MGP Included ',np.nanmean(Included_MGP[tv, :] ) )
    print(' ')
  print('######')

[nan nan]
[0.08886446 0.09291805]
[0.06727953 0.06475333]
[0.08325396 0.08394619]
[0.07749035 0.07630644]
[0.13520154 0.12322697]
[0.06036898 0.07325418]
[0.0729128  0.07309937]
[0.01717596 0.06877755]
[0.07595282 0.07583608]
[0.07562397        nan]
[nan nan]
[0.07479677 0.07520501]
[0.07598067 0.07550024]
[0.08029684 0.0789825 ]
[0.0767372  0.07855772]
[0.07704618 0.07689649]
[0.12475131 0.13071273]
[0.07697833 0.07676351]
[0.07601994 0.07609595]
[0.07892498 0.07865788]
[0.07696626 0.07773884]
[0.07516528 0.07663865]
[0.07838999 0.08142085]
[0.09153424 0.09403608]
[nan nan]
[0.07423087 0.07495897]
[0.07544321 0.0758336 ]
[0.07360027 0.07401369]
[0.07415346 0.07364018]
[0.07757244 0.07718185]
[0.07219318 0.07677229]
[0.07427041 0.0743437 ]
[0.05613421 0.06859622]
[0.07440463 0.07607899]
[0.07462173 0.07397621]
[nan nan]
[0.08160756 0.06486612]
[0.0329151  0.07524177]
[0.07437091 0.07402512]
[0.07647439 0.07589653]
[0.09296989 0.08886229]
[0.07364085 0.07418735]
[       nan 0.05397046]


In [0]:
window = 100
horizons = [22]
kernel = gpytorch.kernels.RBFKernel
Joint_Predictions = [['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

# [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
#                      ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
#                      ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  CI_MGP, Included_MGP = Across_Metals_CI(window, horizon, Target_Variables, kernel, trials=200)
  print('T+',horizon)
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':')
    
    
    print('MGP CI, ',np.nanmean(CI_MGP[tv, :] ))
    print('MGP Included ',np.nanmean(Included_MGP[tv, :] ) )
    print(' ')
  print('######')

[0.34883064 0.37130606]
[nan nan]
[0.46592307 0.37217182]
[0.18178086 0.17926821]
[0.12553972 0.11766804]
[0.2648055  0.23825969]
[0.13645983 0.15191653]
[nan nan]
[0.14662525        nan]
[0.21737635 0.20165443]
[0.36660054 0.36630547]
[0.16942962 0.16352749]
[0.36467975 0.35453954]
[0.20590827 0.19699425]
[0.36193395 0.37511843]
[0.35759053 0.33936122]
[0.30713508 0.29573357]
[0.15533216 0.1518288 ]
[0.22904654 0.25562322]
[0.188077   0.18009979]
[0.33827683 0.3672648 ]
[nan nan]
[0.15939866 0.15844709]
[0.1457387  0.14567196]
[0.20664804 0.22008999]
[0.19232196 0.2068927 ]
[0.5563197  0.53400254]
[0.27130938 0.26890847]
[0.15623589 0.16107683]
[0.2643966  0.24298826]
[0.18275774 0.17957294]
[0.21365719 0.19676715]
[0.26215735 0.22643906]
[0.24836731 0.24595758]
[0.29538262 0.27530482]
[0.3106109  0.32348374]
[0.15259548 0.15503082]
[0.3320739  0.35150215]
[0.18151629 0.19323614]
[0.4041928 0.3937943]
[0.285755  0.2856134]
[0.30717015 0.32280907]
[0.16195403 0.14462182]
[0.27157968 0.